# 01 - Webscraping - Recuperando los datos de la web de ACB.
<hr>
Notebook donde se recupera de la web de la liga ACB los resultados y la inforamción estadistica de los partidos.

La información resumen de cada jornada se encuentra en: https://www.acb.com/resultados-clasificacion/ver/temporada_id/2020

Por cada partido se accede a una nueva página con la información estadística: https://www.acb.com/partido/estadisticas/id/101113

Como resultado se obtiene una estructura de datos tipada, concretamente una colacción de objetos del tipo 'InfoPartidoEquipo'.


In [1]:
# Dependencias:
import requests
from bs4 import BeautifulSoup

# Ejecutamos el notebook donde se encuentran definidas las entidades.
# Lo he sacado a otro notebook porque si queremos utilizarlas en otro notebook se iria repitiendo el código.
%run ./ACB_WebScraping_Entidades.ipynb

In [2]:
class PaginaJornada():
    '''
    Clase que proporciona los métodos para extraer la información de una página de la jornada.
    '''
   
    def __init__(self, temporada, jornada = 1):        
        self._url_jornada = 'https://www.acb.com/resultados-clasificacion/ver/temporada_id/{0}/competicion_id/1/jornada_numero/{1}'        
        self._temporada = temporada        
        self._jornada = jornada
        
        pagina = requests.get (self._url_jornada.format(temporada, jornada))       
        self._soup = BeautifulSoup(pagina.content, 'html.parser')
        
    def recuperarNumeroJornada(self):
        return len(self._soup.select('#listado_resultados_jornada div.elemento'))
        
    def recuperarEstadisticasJornada(self):    
        '''Cada página de jornada contiene el tablón con los emparejamientos.
           Cada emparejamiento contiene un icono con la url de las estadisticas.
           Este método devuelve un listado de las url de estadísticas de la jornada.
        '''
        estadisticas = []
        html_links_estadisticas = self._soup.findAll('a',{'title' : 'Estadísticas'})
        for html_link in html_links_estadisticas:
            estadisticas.append(html_link.get('href'))    
        return estadisticas


In [3]:
class PaginaEstadistica ():
    '''
    Clase que proporciona los métodos para extraer la información de una página de estadísticas.    
    '''
    
    def __init__(self, temporada, jornada, path):        
        self._temporada = temporada
        self._jornada=jornada        
        pagina = requests.get('https://www.acb.com{0}'.format(path))        
        self._soup = BeautifulSoup(pagina.content, 'html.parser')
        
        self._htmlPuntosPorEquipo = ''    
        self._htm_estadisticas=''        
        
    def _recuperaHtmlPuntosPorEquipo(self):
        '''
        Método privado encargado de devolver el htm con el contendido de los puntos por cuarto de cada equipo.
        La información se encuentra en esta cascada de clases ".contenedora_info_principal .info .detalle table tr".
        Los puntos locales están en el segundo tr y los puntos visitantes en el tercer tr.
        '''
        if (self._htmlPuntosPorEquipo == ''):
            self._htmlPuntosPorEquipo = self._soup.select('div .contenedora_info_principal .info .detalle table tr')        
        return self._htmlPuntosPorEquipo    
     
    def _recuperarHtmlEstadisticasEquipo(self):
        '''
        Método privado encargado de devolver el html con el contenido de las estadisticas por equipo.
        Las estadísticas del equipo local están en el primer tr y las estadisticas del equipo visitante en el segundo tr.
        '''
        if (self._htm_estadisticas == ''):
            self._htm_estadisticas = self._soup.select('div .partido tr.totales')                     
        return self._htm_estadisticas
        
    def recuperarPuntosPorCuarto(self, LoV):
        '''
        Recupera los puntos anotados por el equipo local (LoV='L') o visitante (LoV='V').        
        '''
        indice = 1 if (LoV=='L') else 2        
        html = self._recuperaHtmlPuntosPorEquipo()[indice].select('td')                 
        return PuntosEquipoPorCuarto(self._temporada, self._jornada, html[0].text, LoV, html[2].text, html[3].text, html[4].text, html[5].text,html[6].text)
        
    def recuperarEstadisticasPartido(self, LoV):
        '''
        Recupera las estadísticas de equipo local (LoV='L') o visitante (LoV='V').
        '''        
        indice = 0 if (LoV=='L') else 1
        html = self._recuperarHtmlEstadisticasEquipo()[indice].select('td')                 
        return EstadisticasEquipo(self._temporada, self._jornada, '',
                                 html[2].text, html[3].text,  html[4].text,  html[5].text,  html[6].text,
                                 html[7].text, html[8].text,  html[9].text,  html[10].text,
                                 html[22].text)                                 
                                 
    def recuperarInfoPartido(self):    
        infoPartido = []
        infoPartido.append(InfoPartidoEquipo(self.recuperarPuntosPorCuarto('L'), self.recuperarEstadisticasPartido('L')))        
        infoPartido.append(InfoPartidoEquipo(self.recuperarPuntosPorCuarto('V'), self.recuperarEstadisticasPartido('V')))                        
        return infoPartido
        

In [4]:
import pickle

# Recorremos las páginas de cada jornada.
# Recueperamos las url's de estadístcas encontradas.
# Por cada página de estadísiticas extraemos la información que necesitamos.
def recuperarInformacionTemporada(temporada):        
    infoPartidos = []
    print('Recuperando información de la temporada: ', temporada)
    
    page_ultima_jornada = PaginaJornada(temporada)
    total_jornadas_jugadas = page_ultima_jornada.recuperarNumeroJornada()

    for jornada in range(0, total_jornadas_jugadas):
        print('  Recuperando información de la jornada: ', jornada + 1)
        pagina_jornada = PaginaJornada(temporada, jornada + 1)
        url_estadisticas_jornada = pagina_jornada.recuperarEstadisticasJornada()                
    
        for path_url_estadistica in url_estadisticas_jornada:    
            print('    Recuperando información de estadisticas')
            pagina_estadistica = PaginaEstadistica(temporada, jornada + 1, path_url_estadistica)
            infoPartidos.extend (pagina_estadistica.recuperarInfoPartido())
     
    with open('pickle/infoEstadisticasACB_' + str(temporada) + '.pickle', 'wb') as of:     
        pickle.dump(infoPartidos,of)
        of.close()
        

In [7]:
# Al lio!
#for i in range(2015,2020):
recuperarInformacionTemporada(2020)

Recuperando información de la temporada:  2020
  Recuperando información de la jornada:  1
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
  Recuperando información de la jornada:  2
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
  Recuperando información de la jornada:  3
    Recuperando información d

    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
  Recuperando información de la jornada:  20
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
  Recuperando información de la jornada:  21
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
    Recuperando información de estadisticas
  Recuperando información de la jornada:  22
    Recuperando información de estadisticas
    Recuperando información d

En el siguiente notebook daremos la forma que queramos a nuestra colección tipada de objetos...